In [1]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

In [2]:
import keras
from keras.models import model_from_json

Using TensorFlow backend.


In [3]:
!mkdir keras_model

In [4]:
!pwd

/home/ec2-user/SageMaker


In [5]:
model_json = "StockPrediction.json"
json_file = open('/home/ec2-user/SageMaker/keras_model/'+model_json, 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [7]:
model_h5 = "StockPrediction.h5"
loaded_model.load_weights('/home/ec2-user/SageMaker/keras_model/' + model_h5)

Now we are going to try to convert the model from a Keras format to protobuff, so that we can deploy it onto SageMaker

In [8]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

In [9]:
model_version = '1'
export_dir = 'export/Servo/' + model_version

In [10]:
builder = builder.SavedModelBuilder(export_dir)
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"Volatile?": loaded_model.output})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [11]:
from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


In [12]:
# Cheaking the File Structure
!ls export/Servo/1/variables

variables.data-00000-of-00001  variables.index


In [13]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)
import sagemaker

sagemaker_session = sagemaker.Session()

#saving the model to an S3 bucket
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

Creating the entry point python file

In [14]:
!touch train.py

from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

The Python 2 tensorflow images will be soon deprecated and may not be supported for newer upcoming versions of the tensorflow images.
Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


In [20]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium')

Using already existing model: sagemaker-tensorflow-2019-12-23-04-23-49-431


---------------------------------------------------------------------------------------------------------------!CPU times: user 575 ms, sys: 50.2 ms, total: 625 ms
Wall time: 9min 20s


In [21]:
endpoint_name = "sagemaker-tensorflow-2019-12-23-04-23-49-431"
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

In [41]:
import numpy as np

# The sample model expects an input of shape [1,50]

predictor.predict(np.array([[[1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]]]))

{'outputs': {'Volatile?': {'dtype': 1,
   'tensor_shape': {'dim': [{'size': 1}, {'size': 1}]},
   'float_val': [0.4711397886276245]}},
 'model_spec': {'name': 'generic_model',
  'version': {'value': 1},
  'signature_name': 'serving_default'}}